# Load database

In [13]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
from dotenv import load_dotenv

In [14]:
# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Obter caminhos do arquivo csv e do diretório de extração das variáveis de ambiente
clientes_path = os.getenv('CLIENTES_CSV')

In [15]:
# Carregar o CSV em um DataFrame Pandas
df_clientes = pd.read_csv(clientes_path)


**Wrangling**

Vamos realizar a manipulação dos dados para o dashboard, priorizando a obtenção de uma granularidade consistente e aprimoramento da qualidade da base de dados.

In [16]:
df_clientes.head()

,cod_cliente,primeiro_nome,ultimo_nome,email,tipo_cliente,data_inclusao,cpfcnpj,data_nascimento,endereco,cep
0,28,Sabrina,Dias,moreiraemanuelly@example.org,PF,2017-04-03 16:11:00 UTC,357.081.496-39,2006-08-11,Praia de Duarte Vila Piratininga 81327-166 Fer...,95140-704
1,674,Luiz Felipe,Dias,pedroferreira@example.org,PF,2021-02-10 13:27:00 UTC,085.196.374-93,1995-10-11,"Avenida da Rosa, 654 João Paulo Ii 20295449 Nu...",76516-765
2,693,Renan,Dias,ogomes@example.net,PF,2020-01-21 13:12:00 UTC,783.416.059-10,1948-11-19,Jardim de Rodrigues Ipiranga 14161-477 Duarte ...,51779625
3,743,Clara,Dias,rafaelcorreia@example.org,PF,2019-05-06 11:39:00 UTC,589.237.610-95,1978-01-27,"Colônia Thomas Silva, 9 Tupi B 15771-946 Ferna...",19615792
4,769,André,Dias,danilo33@example.org,PF,2017-01-07 14:53:00 UTC,459.608.721-02,1990-08-25,"Rua Correia, 889 Diamante 59123250 Aragão / RS",01672838


In [17]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cod_cliente      998 non-null    int64 
 1   primeiro_nome    998 non-null    object
 2   ultimo_nome      998 non-null    object
 3   email            998 non-null    object
 4   tipo_cliente     998 non-null    object
 5   data_inclusao    998 non-null    object
 6   cpfcnpj          998 non-null    object
 7   data_nascimento  998 non-null    object
 8   endereco         998 non-null    object
 9   cep              998 non-null    object
dtypes: int64(1), object(9)
memory usage: 78.1+ KB


Vamos fazer um type casting da coluna data_inclusao, data_nascimento para o formato de 'datetime64[ns]'

A coluna data_inclusao está formatada com o sufixo "UTC". Para lidar com isso, usaremos a função pd.to_datetime() do pandas, que oferece mais flexibilidade para o parsing de strings de data.

In [18]:
# Converter corretamente as strings de data com o sufixo "UTC" para objetos datetime do pandas
df_clientes['data_inclusao'] = pd.to_datetime(df_clientes['data_inclusao'], utc=True)

# remover a informação de fuso horário, convertendo as datas para o formato local naive datetime
df_clientes['data_inclusao'] = df_clientes['data_inclusao'].dt.tz_localize(None)

In [19]:
df_clientes['data_inclusao'] = df_clientes['data_inclusao'].astype('datetime64[ns]')
df_clientes['data_nascimento'] = df_clientes['data_nascimento'].astype('datetime64[ns]')

Enriquecimento dos dados com novas colunas.

Chaves temporais:

In [20]:
df_clientes['data_inclusao_month'] = df_clientes['data_inclusao'].apply(lambda date: date.strftime('%Y-%m'))
df_clientes['data_inclusao_year']  = df_clientes['data_inclusao'].apply(lambda date: date.strftime('%Y'))

In [21]:
df_clientes['data_nascimento_month'] = df_clientes['data_nascimento'].apply(lambda date: date.strftime('%Y-%m'))
df_clientes['data_nascimento_year']  = df_clientes['data_nascimento'].apply(lambda date: date.strftime('%Y'))

Com os dados manipulados, procederemos à sua persistência em disco, realizaremos o download e, posteriormente, o carregaremos no Google Data Studio.

In [22]:
# Obter caminhos do diretório gold das variáveis de ambiente
gold_path = os.getenv('GOLD_PATH')

In [23]:
df_clientes.to_csv(f'{gold_path}/clientes.csv', sep=',', index=False)